In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CSE272/hw2/

Mounted at /content/drive
/content/drive/MyDrive/CSE272/hw2


In [ ]:
import numpy as np, pandas as pd

In [ ]:
cell_phones_data = pd.read_json('reviews_subset.json',lines=True)
#print(cell_phones_data)
print(cell_phones_data.head())
# dataframe.to_csv('reviews.csv', sep=',', index=False)

   overall  verified   reviewTime      reviewerID        asin  \
0        5      True   08 4, 2014  A24E3SXTC62LJI  7508492919   
1        5      True  02 12, 2014  A269FLZCB4GIPV  7508492919   
2        3      True   02 8, 2014   AB6CHQWHZW4TV  7508492919   
3        2      True   02 4, 2014   A1M117A53LEI8  7508492919   
4        4      True   02 3, 2014  A272DUT8M88ZS8  7508492919   

                  style      reviewerName  \
0  {'Color:': ' Bling'}  Claudia Valdivia   
1                   NaN       sarah ponce   
2                   NaN               Kai   
3                   NaN   Sharon Williams   
4                   NaN   Bella Rodriguez   

                                          reviewText  \
0  Looks even better in person. Be careful to not...   
1  When you don't want to spend a whole lot of ca...   
2  so the case came on time, i love the design. I...   
3  DON'T CARE FOR IT.  GAVE IT AS A GIFT AND THEY...   
4  I liked it because it was cute, but the studs ...   

 

In [ ]:
print(cell_phones_data.dtypes)
print('Minimum rating is: %d' %(cell_phones_data.overall.min()))
print('Maximum rating is: %d' %(cell_phones_data.overall.max()))
print('Number of missing values across columns: \n',cell_phones_data.isnull().sum())


overall             int64
verified             bool
reviewTime         object
reviewerID         object
asin               object
style              object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
vote              float64
image              object
dtype: object
Minimum rating is: 1
Maximum rating is: 5
Number of missing values across columns: 
 overall              0
verified             0
reviewTime           0
reviewerID           0
asin                 0
style             1471
reviewerName         1
reviewText           0
summary              0
unixReviewTime       0
vote              1783
image             2363
dtype: int64


In [ ]:
print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",cell_phones_data.shape[0])
print("Total No of Users   :", len(np.unique(cell_phones_data.reviewerID)))
print("Total No of products  :", len(np.unique(cell_phones_data.asin)))

Total data 
--------------------------------------------------

Total no of ratings : 2372
Total No of Users   : 2144
Total No of products  : 129


In [ ]:
cell_phone =  cell_phones_data[["overall", "reviewerID", "asin"]]
cell_phone.asin = cell_phone.asin.apply(lambda x: x[:-1] if 'X' in x else x)

print(cell_phone)

      overall      reviewerID        asin
0           5  A24E3SXTC62LJI  7508492919
1           5  A269FLZCB4GIPV  7508492919
2           3   AB6CHQWHZW4TV  7508492919
3           2   A1M117A53LEI8  7508492919
4           4  A272DUT8M88ZS8  7508492919
...       ...             ...         ...
2367        5  A249GNBJ30I11K  B000E7UFIG
2368        5  A32EOHLFZYXJEP  B000E7UFIG
2369        5  A2QBJJEKTJ06DR  B000E7UFIG
2370        5  A3OWU1R4LAL8GF  B000E7UFIG
2371        5  A3KDP3BJ7GF5NT  B000E7T7JO

[2372 rows x 3 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
#Analysis of rating given by the user 

no_of_rated_products_per_user = cell_phone.groupby(by='reviewerID')['overall'].count().sort_values(ascending=False)

print(no_of_rated_products_per_user.head())
print(no_of_rated_products_per_user.shape)
print(no_of_rated_products_per_user.describe())

print('\n No of rated product more than 50 per user : {}\n'.format(sum(no_of_rated_products_per_user >= 5)) )

reviewerID
A5JLAU2ARJ0BO     11
A33TED651NAXWU     8
AJMBX04BZWPSD      7
A1F9Z42CFF9IAY     6
A1N5FSCYN4796F     6
Name: overall, dtype: int64
(2144,)
count    2144.000000
mean        1.106343
std         0.518331
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        11.000000
Name: overall, dtype: float64

 No of rated product more than 50 per user : 10



In [ ]:
!pwd
cell_csv_data=pd.read_csv("cell_phone_reviews.csv")
cell_csv_data.head()

cell_csv_data.dtypes

/content/drive/MyDrive/CSE272/hw2


item         object
user         object
rating        int64
timestamp     int64
dtype: object

In [ ]:
!pip install scikit-surprise
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader

import os
from surprise.model_selection import train_test_split

     |████████████████████████████████| 11.8 MB 5.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633700 sha256=bb05580c004c5025606fa016711040a4281c76bf42b663bee0c928cec35a0764
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
#Dropping the Timestamp column

cell_csv_data.drop(['timestamp'], axis=1,inplace=True)

In [ ]:
#Reading the dataset
#print(cell_csv_data)
# reader = Reader(line_format="overall reviewerID asin", sep=' ', rating_scale=(1, 5),skip_lines=1)
reader = Reader(rating_scale=(1, 5),skip_lines=1)
#data = Dataset.load_from_df(cell_phone[['overall', 'reviewerID', 'asin']], reader)
data = Dataset.load_from_df(cell_csv_data, reader)


#Splitting the dataset
trainset, testset = train_test_split(data, test_size=0.2,random_state=10)

# Use user_based true/false to switch between user-based or item-based collaborative filtering
algo = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

# run the trained model against the testset
test_pred = algo.test(testset)

print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)
accuracy.mae(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Item-based Model : Test Set
RMSE: 1.5330
MAE:  1.3545


1.3544558517256402

In [ ]:
# Use user_based true/false to switch between user-based or item-based collaborative filtering
user_basedCF = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': True})
user_basedCF.fit(trainset)

# run the trained model against the testset
user_basedCF_test_pred = user_basedCF.test(testset)

print("User-based Model : Test Set")
accuracy.rmse(user_basedCF_test_pred, verbose=True)
accuracy.mae(user_basedCF_test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
User-based Model : Test Set
RMSE: 1.5343
MAE:  1.3581


1.358054135621013

In [ ]:
from surprise import prediction_algorithms
from surprise import SVD
from surprise import NMF


# min_movie_ratings = 2 #a movie has was rated at least 
# min_user_ratings =  5 #a user rated movies at least

# ratings_flrd_df = ratings_df.groupby("movieId").filter(lambda x: x['movieId'].count() >= min_movie_ratings)
# ratings_flrd_df = ratings_flrd_df.groupby("userId").filter(lambda x: x['userId'].count() >= min_user_ratings)

# print("{0} movies deleted; all movies are now rated at least: {1} times. Old dimensions: {2}; New dimensions: {3}"\
# .format(len(ratings_df.movieId.value_counts()) - len(ratings_flrd_df.movieId.value_counts())\
#         ,min_movie_ratings,ratings_df.shape, ratings_flrd_df.shape ))


algo_SVD = SVD(n_factors = 11)
algo_SVD.fit(trainset)


# Predict ratings for all pairs (i,j) that are NOT in the training set.
SVD_testset = trainset.build_anti_testset()

predictions = algo_SVD.test(SVD_testset)

# subset of the list  predictions
predictions[:5]

[Prediction(uid='9638762632', iid='A3GI4SKQXZOJI1', r_ui=3.706062461726883, est=3.06801183481092, details={'was_impossible': False}),
 Prediction(uid='9638762632', iid='A3Q20EJOGJZGSI', r_ui=3.706062461726883, est=2.9544340237882034, details={'was_impossible': False}),
 Prediction(uid='9638762632', iid='AQVBB9BOQRYVG', r_ui=3.706062461726883, est=3.0647531334477955, details={'was_impossible': False}),
 Prediction(uid='9638762632', iid='A3GHDQATD50SEG', r_ui=3.706062461726883, est=3.1558016936027604, details={'was_impossible': False}),
 Prediction(uid='9638762632', iid='AO9VT2C8L7VHN', r_ui=3.706062461726883, est=3.046051481889364, details={'was_impossible': False})]

In [ ]:
from collections import defaultdict
def get_top_n(predictions, userId, n = 10):# movies_df, ratings_df,
    '''Return the top N (default) movieId for a user,.i.e. userID and history for comparisom
    Args:
    Returns: 
  
    '''
    #Peart I.: Surprise docomuntation
    
    #1. First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    #2. Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_n[uid] = user_ratings[: n ]

    return top_n

print(get_top_n(predictions, 9638762632))

defaultdict(<class 'list'>, {'9638762632': [('A19EOU9LDRSBM9', 3.3220095739823368), ('A38GYFN39HWR0X', 3.259392646645622), ('A18WPFPUYGQS8Y', 3.2462216262880657), ('A2K3SHXTJGR1OP', 3.2449549534090663), ('ARIZCKH460K7R', 3.239692268154324), ('A3EMXJR6UMXITZ', 3.2370250283204056), ('A25BF725P4G1FP', 3.234328024485074), ('A1AQIIG4KD852E', 3.2316764114738317), ('AAM7Z8QYP5JLI', 3.2307414175126072), ('A1VEC12EGN1QDJ', 3.2291440660909263)], '8288853439': [('A1JQ33DNF2Z7UI', 4.478955289348303), ('A1SJHP6301FK1L', 4.478507421219501), ('A3UIEEYEL4MCTB', 4.468699355144998), ('A10860VQHSWUQ7', 4.4592323669800535), ('ACJ8QMHRTF4AZ', 4.45730532870917), ('A3O2DPHARSQGVT', 4.454403862521759), ('A2QRNQBV60LXMN', 4.451614909248241), ('A1RN10T40IZ1GG', 4.4467978158191865), ('A3DCXI6GGZKTCW', 4.440811477607765), ('A2PE1EBFSEYFNZ', 4.436531649808241)], '9638762918': [('A2FSNVND1000RV', 3.70714357754408), ('A1RN10T40IZ1GG', 3.7043542653311903), ('A2PE1EBFSEYFNZ', 3.676212937165679), ('A334IVDF89B6C0', 3.6

In [ ]:
from google.colab import files
uploaded = files.upload()
df4 = pd.read_json("Cell_Phones_and_Accessories_5.json", lines=True)

MessageError: ignored